# Realizando ingeniería de datos para realizar transformación y generación de nuevas variables

## 1.- Importamos las librerías necesarias y preparamos las funciones oportunas.

In [1]:
import os
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
print("Estamos en la carpeta,",folder)

import pandas as pd
import numpy as np
import pickle
import re
import spacy
from nltk.stem.snowball import SnowballStemmer
import datetime
from Utils import functions as f

nlp = spacy.load('es_core_news_lg')

Estamos en la carpeta, d:\Data_science\Javier\Repositorios\Proyecto_tienda_online\src


In [79]:
pd.set_option('display.max_colwidth', None)       # Visualizar todas las columnas
pd.set_option('display.max_rows', None)           # Visualizar todas las filas

Generamos funciones para el tratamiento de textos

In [2]:
def extraer_lemas(texto):
    '''
    Función que recibe un texto como entrada y devuelve una lista de los lemas de las palabras que se encuentran en el texto. 
    
    La lista de lemas resultante solo contiene las palabras que son alfabéticas, es decir, que no contienen caracteres numéricos o especiales.

    - Inputs:

        texto (str): Texto del cual se quieren extraer los lemas.

    - Outputs:

        lemas (list): Lista de los lemas de las palabras alfabéticas que se encuentran en el texto.
    '''
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if token.is_alpha]
    return lemas

def eliminacion_acentos(text):
    '''
    Función que recibe un texto como entrada y devuelve el mismo texto pero sin los caracteres acentuados. 
    
    Para esto, se utiliza una expresión regular y un diccionario con los caracteres a reemplazar.

    - Inputs:

       text (str): Texto del cual se quieren eliminar los acentos.

    - Outputs:

        texto (str): Texto sin los caracteres acentuados.
         '''
    pattern = '[áéíóúÁÉÍÓÚ]'
    replace = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
    return re.sub(pattern, lambda match: replace[match.group()], text)

def spanish_stemmer(x):
    '''
    Función que recibe una cadena de texto como entrada y devuelve el texto procesado con un algoritmo de stemming en español. 
    
    El stemming es una técnica de NLP que busca reducir una palabra a su raíz. 
    
    En este caso, se utiliza el algoritmo SnowballStemmer de la librería NLTK.

    - Inputs:

        x (str): Texto que se quiere procesar con el algoritmo de stemming.

    - Outputs:

        str: Texto procesado con el algoritmo de stemming.
        '''
    stemmer = SnowballStemmer('spanish')
    return " ".join([stemmer.stem(word) for word in x.split()])

def pickled_list(lista,nombre_archivo, carpeta="Utils"):
    '''Generar fichero picke a partir de una lista de tareas. 
    
    Los ficheros se guardan en la carpeta Utils.
    
    - INPUT: 
        - lista (list):         Nombre de la lista a guardar.

        - nombre_archivo (str): Nombre del fichero a generar

        - carpeta: Carpeta donde se va a guardar el fichero .pickle

    - OUTPUT: 
        - lista (.pickle): fichero .pickle donde esta almacenada la información de la lista.

    '''
    ruta_archivo = os.path.join(carpeta, nombre_archivo + ".pickle")

    with open(ruta_archivo, "wb") as archivo_pickle:
            pickle.dump(lista, archivo_pickle)
    return lista


def unpickled_list(lista,nombre_archivo, carpeta="Utils"):
    '''Generar lista de tareas que existen en la carpeta Utils.
    
    Definir tareas nuevas a agregar dentro de la lista en el caso de haber una nueva tarea a añadir.
    
    - INPUT: 
        - lista (list):         Nombre de la lista a guardar.

        - nombre_archivo (str): Nombre del fichero a generar

        - carpeta:              Carpeta donde se va a guardar el fichero .pickle
    
    - OUTPUT: 
        - lista (list):         Lista de productos
    '''
    
    ruta_archivo = os.path.join(carpeta, nombre_archivo + ".pickle")
    with open(ruta_archivo, "rb") as archivo_pickle:
        lista = pickle.load(archivo_pickle)

    return lista

def cargar_listas_desde_pickles(nombres_archivos, carpeta="Utils"):
    listas = {}

    for nombre_archivo in nombres_archivos:
        ruta_archivo = os.path.join(carpeta, nombre_archivo + ".pickle")

        with open(ruta_archivo, "rb") as archivo_pickle:
            lista = pickle.load(archivo_pickle)

        # Cierra el archivo
        archivo_pickle.close()

        # Almacena la lista en el diccionario
        listas[nombre_archivo] = lista

    # Devuelve el diccionario de listas
    return listas

def aplicar_funcion_a_columna(df, listas, nombre_lista, columna="DESCRIPTION"):
    '''
    Función que aplica la función extraer lemas a la columna DESCRIPTION
    La función tiene como parametros de entrada:
    Dataframe
    Diccionario con las listas a tratar
    Columna sobre la que realizar la separación.
    '''
    # Carga la lista desde el diccionario
    lista = listas[nombre_lista]

    # Aplica la función a la columna del DataFrame
    df[nombre_lista] = df[columna].apply(lambda x: any(lematizado in lista for lematizado in extraer_lemas(x)))

    return df

### Obtenemos los datos guardados.

In [3]:
path=r'Data\productos_scrape.csv'

In [4]:
new_path=folder+'\\'+path
new_path

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online\\src\\Data\\productos_scrape.csv'

In [5]:
dataframe=pd.read_csv(new_path)
dataframe.head()

,ID,NAME,INFO,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,MASABOOM - El gran masajeador sexual,"Este es el juguete para todas, todos y todes. ...",https://www.amantis.net/masaboom-el-gran-masaj...,69.99,39.99
1,1,"MASSAJI, Potente masajeador japonés sumergible...",¿Quieres una velada perfecta tras un largo día...,https://www.amantis.net/massaji-potente-masaje...,99.99,44.99
2,2,"DIGIT PRO, dedal vibrador con sujeción",Los dedos siempre han sido grandes aliados de ...,https://www.amantis.net/digit-pro-dedal-vibrad...,79.99,29.99
3,3,"MINI CARNIVAL, mini-masajeador con cuatro cabe...",¿Conoces a MINI+? Puse hoy es tu día de suerte...,https://www.amantis.net/mini-carnival-mini-mas...,34.99,17.99
4,4,LINGÜS - Vibrador para Sexo Oral de amantis,Milenios de sabiduría en el arte del cunniling...,https://www.amantis.net/lingus-vibrador-sexo-o...,79.99,59.99


In [ ]:
len(dataframe)

561

## 2.- Descomponemos las variables *Name* y *Description*.

Vamos a crear 2 nuevas columnas a partir de *Name* y *Description*, donde dejaremos el nombre del producto y su slogan por un lado y por otro la descripción y las características por otro.

En principio son variables que no necesitaremos para estos estudios pero las guardaremos por si hay que hacer algún estudio posterior de las mismas (comparativas de productos similares de esta página web o de otras páginas web, por ejemplo).

#### Generación de slogan.

Primeramente vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos el Name en una nueva columna y varias columnas para ir separando el resto. 

In [6]:
dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)
dataframe['PRODUCT'] = dataframe['PRODUCT'].str.strip()
dataframe['SLOGAN'] = dataframe['SLOGAN'].str.strip()
# dataframe.head()

C:\Users\Javier\AppData\Local\Temp\ipykernel_4660\572804979.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_4660\572804979.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)


#### Separación entre Description y Characteristics.

Primeramente vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos las características en una nueva columna, dejando la Description en la misma. 

**Pendiente**: Hay que hacer una segunda separación de Description en el caso que solo aparezca *características*

In [7]:
dataframe['CHARACTERISTICS'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
dataframe['DESCRIPTION'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
# dataframe.head()


C:\Users\Javier\AppData\Local\Temp\ipykernel_4660\1793188304.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['CHARACTERISTICS'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
C:\Users\Javier\AppData\Local\Temp\ipykernel_4660\1793188304.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['DESCRIPTION'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()


Se observan que hay *\r* dentro del texto generado en las nuevas columnas, por lo que vamos a proceder a sustituir esto por un espacio.

In [8]:
dataframe['CHARACTERISTICS'] = dataframe['CHARACTERISTICS'].str.replace('\r', ' ')
dataframe['DESCRIPTION'] = dataframe['DESCRIPTION'].str.replace('\r', ' ')
# dataframe.head()

In [9]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               561 non-null    int64  
 1   NAME             561 non-null    object 
 2   INFO             561 non-null    object 
 3   LISTA_URL        561 non-null    object 
 4   REGULAR_PRICE    561 non-null    float64
 5   DISCOUNT_PRICE   561 non-null    float64
 6   PRODUCT          561 non-null    object 
 7   SLOGAN           467 non-null    object 
 8   CHARACTERISTICS  501 non-null    object 
 9   DESCRIPTION      561 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 44.0+ KB


Eliminamos *Nombre* e *Info* y sustituimos por *Product*, *Slogan*, *Description* y *Characteristics*.

In [10]:
col_1 = dataframe.pop('PRODUCT')
col_2=dataframe.pop('SLOGAN')
col_3=dataframe.pop('DESCRIPTION')
col_4=dataframe.pop('CHARACTERISTICS')

dataframe.drop(columns=['NAME'],inplace=True)
dataframe.drop(columns=['INFO'],inplace=True)

dataframe.insert(loc= 1 , column= 'PRODUCT', value= col_1)
dataframe.insert(loc= 2 , column= 'SLOGAN', value= col_2)
dataframe.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
dataframe.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
dataframe.head()



,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,MASABOOM,El gran masajeador sexual,"Este es el juguete para todas, todos y todes. ...",Masajeador con forma de micrófonoMaterial: sil...,https://www.amantis.net/masaboom-el-gran-masaj...,69.99,39.99
1,1,MASSAJI,Potente masajeador japonés sumergible de silicona,¿Quieres una velada perfecta tras un largo día...,Masajeador Japonés Massaji Material: Silicona ...,https://www.amantis.net/massaji-potente-masaje...,99.99,44.99
2,2,DIGIT PRO,dedal vibrador con sujeción,Los dedos siempre han sido grandes aliados de ...,Dedal vibradorMaterial: silicona de grado médi...,https://www.amantis.net/digit-pro-dedal-vibrad...,79.99,29.99
3,3,MINI CARNIVAL,mini_masajeador con cuatro cabezales,¿Conoces a MINI+? Puse hoy es tu día de suerte...,Mini masajeador con cuatro cabezalesCabezales ...,https://www.amantis.net/mini-carnival-mini-mas...,34.99,17.99
4,4,LINGÜS,Vibrador para Sexo Oral de amantis,Milenios de sabiduría en el arte del cunniling...,Material: silicona + abs10 modos de vibraciónT...,https://www.amantis.net/lingus-vibrador-sexo-o...,79.99,59.99


Pasamos a un fichero .csv para guardarlo

In [11]:
df_product=dataframe.iloc[:,:6]
df_product.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,MASABOOM,El gran masajeador sexual,"Este es el juguete para todas, todos y todes. ...",Masajeador con forma de micrófonoMaterial: sil...,https://www.amantis.net/masaboom-el-gran-masaj...
1,1,MASSAJI,Potente masajeador japonés sumergible de silicona,¿Quieres una velada perfecta tras un largo día...,Masajeador Japonés Massaji Material: Silicona ...,https://www.amantis.net/massaji-potente-masaje...
2,2,DIGIT PRO,dedal vibrador con sujeción,Los dedos siempre han sido grandes aliados de ...,Dedal vibradorMaterial: silicona de grado médi...,https://www.amantis.net/digit-pro-dedal-vibrad...
3,3,MINI CARNIVAL,mini_masajeador con cuatro cabezales,¿Conoces a MINI+? Puse hoy es tu día de suerte...,Mini masajeador con cuatro cabezalesCabezales ...,https://www.amantis.net/mini-carnival-mini-mas...
4,4,LINGÜS,Vibrador para Sexo Oral de amantis,Milenios de sabiduría en el arte del cunniling...,Material: silicona + abs10 modos de vibraciónT...,https://www.amantis.net/lingus-vibrador-sexo-o...


In [11]:
df_product.to_csv('./Data/productos.csv',header=True,index=False)
# df_product.to_csv('./Data/tags_scrape.csv',header=True,index=False)

## 3.- Generamos el fichero de *precios* a partir de df_product.

Para ello vamos a partir del anterior dataframe, donde dejaremos los datos de *ID*, *REGULAR_PRICE* y *DISCOUNT_PRICE*.

Este ID, lo convertiremos en el ID_product y asignaremos un nuevo ID, ya que vamos a agregar una nueva columna con la fecha en la cual se guarda este fichero (en principio, como es un proceso continuo debería de corresponder con la fecha de descarga).

In [40]:
dataframe=pd.read_csv(new_path)
date=str(datetime.datetime.today().strftime('%y%m%d'))
col_1 = dataframe.pop('REGULAR_PRICE')
col_2=dataframe.pop('DISCOUNT_PRICE')
col_3=dataframe['ID']
# col_4=dataframe['ID']

dataframe.insert(loc= 1 , column= 'ID_PRODUCT', value= col_3)
dataframe.insert(loc= 2 , column= 'REGULAR_PRICE', value= col_1)
dataframe.insert(loc= 3 , column= 'DISCOUNT_PRICE', value= col_2)

df_prices=dataframe.iloc[:,:4]
df_prices['FECHA']=date

df_prices.head()

,ID,ID_PRODUCT,REGULAR_PRICE,DISCOUNT_PRICE,FECHA
0,0,0,69.99,39.99,231012
1,1,1,99.99,44.99,231012
2,2,2,79.99,29.99,231012
3,3,3,34.99,17.99,231012
4,4,4,79.99,59.99,231012


In [41]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              561 non-null    int64  
 1   ID_PRODUCT      561 non-null    int64  
 2   REGULAR_PRICE   561 non-null    float64
 3   DISCOUNT_PRICE  561 non-null    float64
 4   FECHA           561 non-null    object 
dtypes: float64(2), int64(2), object(1)
memory usage: 22.0+ KB


Vamos a guardar el dataframe en un .csv, para lo cual vamos a agregar una fecha en el nombre para saber cuándo se subio el fichero.

In [47]:
name='precios'
new_name=name+'_'+date
new_name
folder_to_upload='./Data/'+new_name+'.csv'
folder_to_upload
df_prices.to_csv(folder_to_upload,header=True,index=False)

## 4.- Adecuamos el campo **Date** y el campo **Users**.

In [49]:
comentarios=r'\Data\comentarios_scrape.csv'

In [50]:
folder
new_path =folder+comentarios
df_comments=pd.read_csv(new_path)

In [51]:
df_comments.head()

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT
0,0,1,"miércoles 25 enero, 2023",5,Sara,Es un vibrador discreto y eficaz. Es uno de mi...
1,1,1,"lunes 09 enero, 2023",5,Jesús,el vibrador de cabecera de mi mujer. Después d...
2,2,1,"sábado 10 diciembre, 2022",3,Sara,Tenía uno de otra tienda y compré este porque ...
3,3,1,"martes 04 octubre, 2022",5,Sara,"Es una buena compra. Muy agradable, con difere..."
4,4,1,"jueves 07 julio, 2022",5,Álvaro,Buen juguete más centrado en los precalentamie...


### a.- Date:

Este campo en el fichero de comentarios es de tipo *string*, dado que no lo reconoce como date.

Hay que solucionarlo, para lo cual elimineramos el día de la semana del mismo dejando el resto del texto y convirtiéndolo en tipo date.


In [52]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10479 entries, 0 to 10478
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10479 non-null  int64 
 1   ID          10479 non-null  int64 
 2   DATE        10479 non-null  object
 3   RATIO       10479 non-null  int64 
 4   USERS       10469 non-null  object
 5   COMMENT     10479 non-null  object
dtypes: int64(3), object(3)
memory usage: 491.3+ KB


De la columna *DATE* extraigo los valores de *DAY*, *MONTH* y *YEAR*.

Con la columna *MONTH* hago un mapeo para convertir en el número correspondiente del mes y así poder generar la nueva columna *DATE* con estos  datos.

In [53]:
df_comments['DAY']=df_comments['DATE'].str.split(' ').str.get(1).astype('Int64')
df_comments['MONTH']=df_comments['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
df_comments['YEAR']=df_comments['DATE'].str.split(' ').str.get(-1).astype('Int64')
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 
df_comments['MONTH']=df_comments['MONTH'].map(dm_mapping)
df_comments.head()

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT,DAY,MONTH,YEAR
0,0,1,"miércoles 25 enero, 2023",5,Sara,Es un vibrador discreto y eficaz. Es uno de mi...,25,1,2023
1,1,1,"lunes 09 enero, 2023",5,Jesús,el vibrador de cabecera de mi mujer. Después d...,9,1,2023
2,2,1,"sábado 10 diciembre, 2022",3,Sara,Tenía uno de otra tienda y compré este porque ...,10,12,2022
3,3,1,"martes 04 octubre, 2022",5,Sara,"Es una buena compra. Muy agradable, con difere...",4,10,2022
4,4,1,"jueves 07 julio, 2022",5,Álvaro,Buen juguete más centrado en los precalentamie...,7,7,2022


Convierto la columna *DATE* en datetime con los datos de las 3 últimas columnas.

In [54]:
df_comments['DATE'] = pd.to_datetime(df_comments.iloc[:,-3:])
df_comments.tail()

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT,DAY,MONTH,YEAR
10474,10925,574,2019-11-06,2,Maria,"No está mal, pero puede resultar frustrante. E...",6,11,2019
10475,10926,574,2018-04-03,3,Miguel Ángel,"Buen tacto, aunque por su forma hay que estar ...",3,4,2018
10476,10927,574,2018-03-12,5,j.r.d.,"Tacto muy suave, no lo hemos provado mucho per...",12,3,2018
10477,10928,574,2018-02-08,3,Amaya,"El tacto es genial, muy agradable y suave. La ...",8,2,2018
10478,10929,575,2019-10-08,1,Maria,OJO con este. Lo compramos porque el ARO THIN ...,8,10,2019


Cabe la posibilidad de eliminar las 3 últimas columnas. 

Esto lo valoraremos después cuando hagamos visualizaciones.

De momento lo dejaremos comentado.

In [55]:
df_comments=df_comments.iloc[:,:-3]
df_comments

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT
0,0,1,2023-01-25,5,Sara,Es un vibrador discreto y eficaz. Es uno de mi...
1,1,1,2023-01-09,5,Jesús,el vibrador de cabecera de mi mujer. Después d...
2,2,1,2022-12-10,3,Sara,Tenía uno de otra tienda y compré este porque ...
3,3,1,2022-10-04,5,Sara,"Es una buena compra. Muy agradable, con difere..."
4,4,1,2022-07-07,5,Álvaro,Buen juguete más centrado en los precalentamie...
...,...,...,...,...,...,...
10474,10925,574,2019-11-06,2,Maria,"No está mal, pero puede resultar frustrante. E..."
10475,10926,574,2018-04-03,3,Miguel Ángel,"Buen tacto, aunque por su forma hay que estar ..."
10476,10927,574,2018-03-12,5,j.r.d.,"Tacto muy suave, no lo hemos provado mucho per..."
10477,10928,574,2018-02-08,3,Amaya,"El tacto es genial, muy agradable y suave. La ..."


### b. Users:

Vamos a discriminar en función de si un nombre de usuario a escrito un comentario sobre un producto, si vuelve a aparecer este nombre se genere un nombre nuevo, que será el mismo con un dígito para distinguirlos.

Es cierto que esta forma de presentar los datos a través de la página web aumenta el anonimato de las personas que lo compran, ya que lo habitual es que se repitan nombres sin coincidir con un mismo usuario (¿cuántos José hay en España?¿y en el mundo?)

No es posible hacer un seguimiento de qué productos compran las personas de este modo, pero vamos a realizar un "simulacro" de cómo se debería de realizar.

El modo correcto sería usando id de usuarios, lo que solo es posible por parte de la empresa o si se utilizarán nicks para que aparecieran los usuarios que realizan los comentarios por su nick.

In [56]:
nombre_count = {}
count = {}

for i, row in df_comments.iterrows():
    id = row['ID']
    nombre = row['USERS']
    
    if id not in count:
        count[id] = {}
        
    if nombre in count[id]:
        count[id][nombre] += 1
        nueva_nombre = f"{nombre}_{count[id][nombre]}"
        df_comments.loc[i, 'USERS'] = nueva_nombre
    else:
        count[id][nombre] = 1



In [57]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10479 entries, 0 to 10478
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  10479 non-null  int64         
 1   ID          10479 non-null  int64         
 2   DATE        10479 non-null  datetime64[ns]
 3   RATIO       10479 non-null  int64         
 4   USERS       10471 non-null  object        
 5   COMMENT     10479 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 491.3+ KB


Verificamos.... 

Lo hacemos realizando sustituiciones de nombres y nombres_digito diferenciador para nombres comunes.

In [58]:
df_comments[df_comments['USERS']=='Maria_2']

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT
525,525,10,2020-06-03,5,Maria_2,Me gusta mucho la marca Desliz! Lo uso con mis...
1095,1095,15,2021-11-29,5,Maria_2,"Buenos días, tengo una consulta. Para desinfec..."
1439,1439,26,2013-07-17,3,Maria_2,Nunca supe ser breve así q por lo menos seré o...
3763,4008,126,2020-12-15,5,Maria_2,Muy práctica para guardar los juguetitos indiv...
4537,4782,136,2019-06-02,5,Maria_2,Enamorada me hallo. Compré por probar y me ha ...
5181,5507,172,2016-04-16,5,Maria_2,"UUfff sin palabras te deja la maquineta ,,,, T..."
5554,6005,203,2018-09-18,5,Maria_2,Tiene una vibración muy potente y hace mucho r...
5687,6138,204,2019-02-11,5,Maria_2,aún no lo he utilizado porque no he tenido oca...
6259,6710,205,2009-09-15,5,Maria_2,Lleva varios meses en casa y ahora ya si puedo...
7180,7631,279,2014-10-22,5,Maria_2,"Genial, cunde muchísimo, aroma, textura y sabo..."


Vamos a generar un nuevo dataframe con los datos de los usuarios para pasarlos posteriormente a una base de datos relacional y que así esté lo más atomizada posible.

In [59]:
lista_users=df_comments['USERS'].unique()
len(lista_users)

2021

In [60]:
lista_users=df_comments['USERS'].unique()                  
mivalor = [ x for x in range(len(lista_users))]             
lista_users=list(lista_users)                                
lista_users_code = {k: v for k, v in zip(lista_users, mivalor)}   
print(lista_users_code)
df_comments['ID_USERS']= df_comments['USERS'].map(lista_users_code)
df_comments.head()

{'Sara': 0, 'Jesús': 1, 'Sara_2': 2, 'Sara_3': 3, 'Álvaro': 4, 'Yenifer': 5, 'Mirian': 6, 'Paula': 7, 'María': 8, 'María de Nieves': 9, 'rocio': 10, 'Candela': 11, 'Soraya': 12, 'Ricardo': 13, 'Alonso': 14, 'Lidia': 15, 'Clara': 16, 'Ana': 17, 'Joaquín': 18, 'VIOLETA': 19, 'David': 20, 'Luis': 21, 'Sonia': 22, 'Gema': 23, 'Angeles': 24, 'Javier': 25, 'Marta': 26, 'Paula_2': 27, 'Beatriz': 28, 'Jim': 29, 'Oscar': 30, 'Silvia': 31, 'Jesús Armand': 32, 'Angel': 33, 'Natalia': 34, 'Mario': 35, 'MARINA': 36, 'Christian': 37, 'Marina': 38, 'Gloria': 39, 'Antía': 40, 'Rosa': 41, 'Virginia': 42, 'Ariadna': 43, 'MARTA': 44, 'Fer': 45, 'Andrea': 46, 'Sacha': 47, 'Sergio': 48, 'Miguel': 49, 'Niko': 50, 'Javier_2': 51, 'Daniel': 52, 'BEATRIZ': 53, 'yolanda': 54, 'Bryan': 55, 'Manuel': 56, 'Úrsula': 57, 'Markos': 58, 'ainho': 59, 'Ruben': 60, 'antonio': 61, 'Juan José': 62, 'Sacha_2': 63, 'Laura': 64, 'DANYEL': 65, 'Claudia': 66, 'Sergio_2': 67, 'Javier_3': 68, 'Irene': 69, 'Jonathan': 70, 'Veronic

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT,ID_USERS
0,0,1,2023-01-25,5,Sara,Es un vibrador discreto y eficaz. Es uno de mi...,0
1,1,1,2023-01-09,5,Jesús,el vibrador de cabecera de mi mujer. Después d...,1
2,2,1,2022-12-10,3,Sara_2,Tenía uno de otra tienda y compré este porque ...,2
3,3,1,2022-10-04,5,Sara_3,"Es una buena compra. Muy agradable, con difere...",3
4,4,1,2022-07-07,5,Álvaro,Buen juguete más centrado en los precalentamie...,4


In [61]:
USERS=pd.DataFrame()
USERS['ID_USERS']=df_comments['ID_USERS']
USERS['USERS']=df_comments['USERS']
USERS.drop_duplicates(subset='ID_USERS', keep='first',inplace=True)
USERS.head()
len(USERS)

2021

Reestructuramos *df_comments*.

In [62]:
col = df_comments.pop('ID_USERS')

df_comments.drop(columns=['USERS'],inplace=True)

df_comments.insert(loc= 4 , column= 'ID_USERS', value= col)
df_comments.head()

,Unnamed: 0,ID,DATE,RATIO,ID_USERS,COMMENT
0,0,1,2023-01-25,5,0,Es un vibrador discreto y eficaz. Es uno de mi...
1,1,1,2023-01-09,5,1,el vibrador de cabecera de mi mujer. Después d...
2,2,1,2022-12-10,3,2,Tenía uno de otra tienda y compré este porque ...
3,3,1,2022-10-04,5,3,"Es una buena compra. Muy agradable, con difere..."
4,4,1,2022-07-07,5,4,Buen juguete más centrado en los precalentamie...


In [63]:
df_comments.to_csv('./Data/comentarios.csv',header=True,index=False)           # Tengo que generar el path correcto

USERS.to_csv('./Data/usuarios.csv',header=True,index=False)

## 5.- Generamos los tags para procesado de datos.

Definimos la lista de tags que queremos asignar. 

Esto lo haremos generando listas con los nombres lemmatizados.

También hay que dejar el texto utilizado en minúscula.

In [12]:
juguetes=['dildo','plug','vibrador','masturbador','cabezal','estimulador','plugs','bolas chinas','funda extensora','bomba de vacio']
BDSM=['latex','bdsm','arnes','strap','cera','ligadura','cuerda','cuero','sumision','dominacion','latigo','watenberg','posicionador','mordaza']
muebles=['columpio','sillon','sillones']
lenceria=['body','panties','corse',]
anal=['anal']
masculino=['hombre','masculino']
femenino=['mujer','femenino','vaginal','clitoris','dildo']
amenities=['lubricante','limpiador','preservativo','condon']
posicionador=[]                         # Falta por agregar parametros 

Vamos a guardar las diferentes listas que hemos determinado en ficheros para posteriormente, cargarlas y realizar la generación de variables a partir de ellas.

In [64]:
f.pickled_list(juguetes,'juguetes', carpeta="Utils")
f.pickled_list(BDSM,'BDSM', carpeta="Utils")
f.pickled_list(muebles,'muebles', carpeta="Utils")
f.pickled_list(lenceria,'lenceria', carpeta="Utils")
f.pickled_list(anal,'anal', carpeta="Utils")
f.pickled_list(masculino,'masculino', carpeta="Utils")
f.pickled_list(femenino,'femenino', carpeta="Utils")
f.pickled_list(amenities,'amenities', carpeta="Utils")
# f.pickled_list(lista,nombre_archivo, carpeta="Utils")
# f.pickled_list(lista,nombre_archivo, carpeta="Utils")

NameError: name 'juguetes' is not defined

In [13]:
path=r'\Data\productos.csv'
file=folder+path

In [14]:
df_engineer=pd.read_csv(file)
df_engineer.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,MASABOOM,El gran masajeador sexual,"Este es el juguete para todas, todos y todes. ...",Masajeador con forma de micrófonoMaterial: sil...,https://www.amantis.net/masaboom-el-gran-masaj...
1,1,MASSAJI,Potente masajeador japonés sumergible de silicona,¿Quieres una velada perfecta tras un largo día...,Masajeador Japonés Massaji Material: Silicona ...,https://www.amantis.net/massaji-potente-masaje...
2,2,DIGIT PRO,dedal vibrador con sujeción,Los dedos siempre han sido grandes aliados de ...,Dedal vibradorMaterial: silicona de grado médi...,https://www.amantis.net/digit-pro-dedal-vibrad...
3,3,MINI CARNIVAL,mini_masajeador con cuatro cabezales,¿Conoces a MINI+? Puse hoy es tu día de suerte...,Mini masajeador con cuatro cabezalesCabezales ...,https://www.amantis.net/mini-carnival-mini-mas...
4,4,LINGÜS,Vibrador para Sexo Oral de amantis,Milenios de sabiduría en el arte del cunniling...,Material: silicona + abs10 modos de vibraciónT...,https://www.amantis.net/lingus-vibrador-sexo-o...


#### Pasamos todos los caracteres a minúsculas y eliminamos los acentos.

In [17]:
df_engineer['SLOGAN'] = df_engineer['SLOGAN'].str.lower()
df_engineer['DESCRIPTION'] = df_engineer['DESCRIPTION'].str.lower()
# df_engineer['SLOGAN'] = df_engineer['SLOGAN'].apply(f.eliminacion_acentos)              # Esto da error, debe de ser porque hay NaN en el campo
df_engineer['DESCRIPTION'] = df_engineer['DESCRIPTION'].apply(f.eliminacion_acentos)

df_engineer.head(2)

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,MASABOOM,el gran masajeador sexual,"este es el juguete para todas, todos y todes. ...",Masajeador con forma de micrófonoMaterial: sil...,https://www.amantis.net/masaboom-el-gran-masaj...
1,1,MASSAJI,potente masajeador japonés sumergible de silicona,¿quieres una velada perfecta tras un largo dia...,Masajeador Japonés Massaji Material: Silicona ...,https://www.amantis.net/massaji-potente-masaje...


Vamos a ver como cargamos cada fichero para proceder a la generación de *tags*.


In [20]:
nombre_listas=['amenities','anal','BDSM','femenino','masculino','juguetes','lenceria','muebles']

In [21]:
listas = f.cargar_listas_desde_pickles(nombre_listas)

In [22]:
listas

{'amenities': ['lubricante', 'limpiador', 'preservativo', 'condon'],
 'anal': ['anal'],
 'BDSM': ['latex',
  'bdsm',
  'arnes',
  'strap',
  'cera',
  'ligadura',
  'cuerda',
  'cuero',
  'sumision',
  'dominacion',
  'latigo',
  'watenberg',
  'posicionador',
  'mordaza'],
 'femenino': ['mujer', 'femenino', 'vaginal', 'clitoris', 'dildo'],
 'masculino': ['hombre', 'masculino'],
 'juguetes': ['dildo',
  'plug',
  'vibrador',
  'masturbador',
  'cabezal',
  'estimulador',
  'plugs',
  'bolas chinas',
  'funda extensora',
  'bomba de vacio'],
 'lenceria': 'lenceria',
 'muebles': ['chaise', 'columpio', 'moqueta', 'silla', 'sillon', 'sillones']}

In [27]:
for nombre_lista in listas:
    df_engineer = f.aplicar_funcion_a_columna(df_engineer,listas, nombre_lista)

In [28]:
df_engineer.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,amenities,anal,BDSM,femenino,masculino,juguetes,lenceria,muebles
0,0,MASABOOM,el gran masajeador sexual,"este es el juguete para todas, todos y todes. ...",Masajeador con forma de micrófonoMaterial: sil...,https://www.amantis.net/masaboom-el-gran-masaj...,False,False,False,False,False,True,True,False
1,1,MASSAJI,potente masajeador japonés sumergible de silicona,¿quieres una velada perfecta tras un largo dia...,Masajeador Japonés Massaji Material: Silicona ...,https://www.amantis.net/massaji-potente-masaje...,True,False,False,False,False,True,True,False
2,2,DIGIT PRO,dedal vibrador con sujeción,los dedos siempre han sido grandes aliados de ...,Dedal vibradorMaterial: silicona de grado médi...,https://www.amantis.net/digit-pro-dedal-vibrad...,False,False,False,False,False,True,True,False
3,3,MINI CARNIVAL,mini_masajeador con cuatro cabezales,¿conoces a mini+? puse hoy es tu dia de suerte...,Mini masajeador con cuatro cabezalesCabezales ...,https://www.amantis.net/mini-carnival-mini-mas...,False,False,False,False,False,True,True,False
4,4,LINGÜS,vibrador para sexo oral de amantis,milenios de sabiduria en el arte del cunniling...,Material: silicona + abs10 modos de vibraciónT...,https://www.amantis.net/lingus-vibrador-sexo-o...,False,False,False,True,False,True,True,False


In [63]:
# juguetes=unpickled_list('juguetes', carpeta="Utils")
# juguetes

['dildo',
 'plug',
 'vibrador',
 'masturbador',
 'cabezal',
 'estimulador',
 'plugs',
 'bolas chinas',
 'funda extensora',
 'bomba de vacio']

Generamos las variables nuevas a partir de las listas con las keywords.

In [58]:
# df_engineer['lenceria'] = df_engineer['Description'].apply(lambda x: any(lematizado in lenceria for lematizado in extraer_lemas(x)))
# df_engineer['juguetes'] = df_engineer['Description'].apply(lambda x: any(lematizado in juguetes for lematizado in extraer_lemas(x)))
# df_engineer['BDSM'] = df_engineer['Description'].apply(lambda x: any(lematizado in BDSM for lematizado in extraer_lemas(x)))
# df_engineer['muebles'] = df_engineer['Description'].apply(lambda x: any(lematizado in muebles for lematizado in extraer_lemas(x)))
# df_engineer['anal'] = df_engineer['Description'].apply(lambda x: any(lematizado in anal for lematizado in extraer_lemas(x)))
# df_engineer['masculino'] = df_engineer['Description'].apply(lambda x: any(lematizado in masculino for lematizado in extraer_lemas(x)))
# df_engineer['femenino'] = df_engineer['Description'].apply(lambda x: any(lematizado in femenino for lematizado in extraer_lemas(x)))
# df_engineer['amenities'] = df_engineer['Description'].apply(lambda x: any(lematizado in amenities for lematizado in extraer_lemas(x)))

# df_engineer['otros'] = ~(df_engineer['lenceria'] | df_engineer['juguetes'] | df_engineer['BDSM']| df_engineer['muebles'] | df_engineer['anal']| df_engineer['masculino'] | df_engineer['femenino'] | df_engineer['amenities']).any()


# df_engineer['Description'] = df_engineer['Description'].apply(spanish_stemmer)

# df_engineer

Hacemos una visualización de las cantidades que hay en cada una de las nuevas columnas para después ver si han realizado bien la separación o no y volver a definirlas.

Haremos después una visualización uno a uno de los diferentes tags generados para verificar si están bien definidos y en el caso que haya que realizar una modificación generar una nueva lista para incluirla en las iniciales.

##### Lencería

In [25]:
df_engineer['lenceria'].value_counts()

False    456
True      95
Name: lenceria, dtype: int64

In [ ]:
df_engineer[df_engineer['lenceria']==True]

Hay vibradores dentro de esta lista porque en el texto está la palabra body.

In [ ]:
df_engineer[df_engineer['lenceria']==False]

En el caso que tengamos que añadir algún otro texto a cada uno de los componentes del diccionario, lo aplicaremos y realizamos un pickle de éste.

Extraemos los datos de la lista correspondiente.

In [35]:
lenceria=unpickled_list('lenceria','lenceria', carpeta="Utils")
lenceria

['babydoll',
 'body',
 'camiseta',
 'corse',
 'corset',
 'diadema',
 'encaje',
 'falda',
 'joyeria',
 'lenceria',
 'lencero',
 'malla',
 'media',
 'panties',
 'pantis',
 'ropa',
 'sujetador',
 'vestido']

Generamos nuevos tags que incluimos dentro de la lista.

In [36]:
lenceria_new=[]                         # Aquí agregamos los nuevos tags para incluir
lenceria+=lenceria_new          
lenceria.sort()     
lenceria

['babydoll',
 'body',
 'camiseta',
 'corse',
 'corset',
 'diadema',
 'encaje',
 'falda',
 'joyeria',
 'lenceria',
 'lencero',
 'malla',
 'media',
 'panties',
 'pantis',
 'ropa',
 'sujetador',
 'vestido']

Volvemos a 'picklear' el fichero.

In [37]:
pickled_list('lenceria','lenceria')

'lenceria'

In [30]:
df_engineer['lenceria'] = df_engineer['Description'].apply(lambda x: any(lematizado in lenceria for lematizado in extraer_lemas(x)))

In [31]:
df_engineer['lenceria'].value_counts()

False    462
True      95
Name: lenceria, dtype: int64

In [ ]:
df_engineer[df_engineer['lenceria']==True]

Hay varios productos que no deben de estar, tienen vibrador, succionador y mordaza.

Estos productos deberían de cambiar de True a False.

In [ ]:
df_engineer[df_engineer['lenceria']==False]

Qué hacemos con los collares y las cadenas para pezones?

##### Juguetes

In [32]:
df_engineer['juguetes'].value_counts()

False    311
True     246
Name: juguetes, dtype: int64

In [ ]:
df_engineer[df_engineer['juguetes']==True]

En este punto se han incluido como juguetes productos que no lo son como amenities (limpiadores, para guardar) o arneses.

In [ ]:
df_engineer[df_engineer['juguetes']==False]

Se ve que hay bastantes productos mal asignados, principalmente anillas para penes, bombas de succion, hay plugs que no se han asignado bien, bolas chinas,...

##### BDSM

In [33]:
df_engineer['BDSM'].value_counts()

False    411
True     146
Name: BDSM, dtype: int64

In [ ]:
df_engineer[df_engineer['BDSM']==True]

In [ ]:
df_engineer[df_engineer['BDSM']==False]

##### Muebles

In [34]:
df_engineer['muebles'].value_counts()

False    552
True       5
Name: muebles, dtype: int64

In [ ]:
df_engineer[df_engineer['muebles']==True]

Hay que cambiar Black Line

In [ ]:
df_engineer[df_engineer['muebles']==False]

In [56]:
muebles_new=['chaise','silla','moqueta']          # Pendiente de agregar, no puedo usar puerta
muebles+=muebles_new
muebles.sort()
muebles

['chaise', 'columpio', 'moqueta', 'silla', 'sillon', 'sillones']

Agregando los nuevos valores

In [36]:
df_engineer['muebles'] = df_engineer['Description'].apply(lambda x: any(lematizado in muebles for lematizado in extraer_lemas(x)))
df_engineer['muebles'].value_counts()

False    543
True      14
Name: muebles, dtype: int64

In [ ]:
df_engineer[df_engineer['muebles']==True]

Hay que eliminar los dildos y plugs y agregar el soporte para puertas

##### Anal

In [24]:
df_engineer['anal'].value_counts()

False    453
True     123
Name: anal, dtype: int64

In [ ]:
df_engineer[df_engineer['anal']==True]

In [ ]:
df_engineer[df_engineer['anal']==False]

##### Masculino

In [27]:
df_engineer['masculino'].value_counts()

False    525
True      51
Name: masculino, dtype: int64

In [ ]:
df_engineer[df_engineer['masculino']==True]

In [ ]:
df_engineer[df_engineer['masculino']==False]

##### Femenino

In [38]:
df_engineer['femenino'].value_counts()

False    367
True     184
Name: femenino, dtype: int64

In [ ]:
df_engineer[df_engineer['femenino']==True]

Hay juguetes masculinos que se han incluido aquí dado que tienen la palabra mujer. 

Estos son las muñecas para hombres. 

In [ ]:
df_engineer[df_engineer['femenino']==False]

##### Amenities

In [39]:
df_engineer['amenities'].value_counts()

False    397
True     154
Name: amenities, dtype: int64

In [ ]:
amenities=['lubricante','limpiador','preservativo','condon']            #Tengo que aplicar primero la generación de las variables

In [ ]:
df_engineer[df_engineer['amenities']==True]

In [ ]:
df_engineer[df_engineer['amenities']==False]

##### Otros (Esta columna tengo pendiente de crear)

In [40]:
df_engineer['otros'].value_counts()

KeyError: 'otros'

In [37]:
df_engineer[df_engineer['otros']==True]

,Name,Description,lenceria,juguetes,BDSM,muebles,anal,masculino,femenino,amenities,otros


In [ ]:
df_engineer[df_engineer['otros']==False]

Guardamos el fichero con la tabla de *Tags*, para ello tomamos todos los campos salbo **'DESCRIPTION'** Y **'SLOGAN'**

In [41]:
df_engineer.drop(columns=['SLOGAN','DESCRIPTION'],inplace=True)
df_engineer.head

<bound method NDFrame.head of       ID                                            PRODUCT  amenities   anal  \
0      0            Desliz! Lubricante íntimo de agua 100ml       True  False   
1      1                                            FOXTAIL       True   True   
2      2                                             LIZO 2      False  False   
3      3                          Bacanal FORTE TARRO 200ml       True   True   
4      4   Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml      False  False   
..   ...                                                ...        ...    ...   
546  571                                             BRILLI      False  False   
547  572                                  Glup! Plug grande       True   True   
548  573  Conjunto de encaje + collar y muñequeras con c...      False  False   
549  574                           Cabezal para masaje_sexy      False  False   
550  575                                           MS_DIGIT      False   True  

In [42]:
df_engineer.to_csv('./Data/tags.csv',header=True,index=False)